<a href="https://colab.research.google.com/github/Logicus03/Image-Classification-ImageAI/blob/master/Image_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Recognition using Tensorflow

Using Tutorial created by [HobbieCode](https://www.hobbiecode.com/reconocimiento-de-imagenes-con-python/)

In [1]:
# Install main packages

!pip install opencv-python
!pip install keras

In [2]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
1.15.2


In [3]:
!pip install imageai --upgrade


     |████████████████████████████████| 184kB 6.7MB/s 


By using ImageAI library, create the model we're going to train

#**Files Management**

Download files

In [4]:
import requests
from requests.adapters import HTTPAdapter
from requests import Session, exceptions
from requests.packages.urllib3.util.retry import Retry
from tqdm import tqdm
import http
import os

http.client.HTTPConnection.debuglevel = 0

In [5]:
# https://blog.ian.stapletoncordas.co/2014/12/retries-in-requests.html
def requests_retry_session( 
    retries = 10,
    backoff_factor = 0.3,    
    status_forcelist = [104, 413, 429, 500, 502, 503, 504 ],
    method_whitelist = ["HEAD", "GET", "PUT", "DELETE", "OPTIONS", "TRACE"]
    ):
  
    session = Session()
    retry = Retry(
        total = retries, read = retries, connect = retries, backoff_factor = backoff_factor, 
        status_forcelist  = status_forcelist, method_whitelist  = method_whitelist
    )

    adapter = HTTPAdapter( max_retries = retry )
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session
    


In [6]:

def download_zip_file(url, file_name):

  ret_val = False
  block_size = 1024
  initial_pos = 0
  mode = 'wb'

  try:
    response = requests_retry_session().get(url, timeout = 5, stream=True)

  except Exception as x:
    time.sleep(20)
    
  else:
    
    if ( response.status_code == 200 ):
      file_size = int(response.headers.get('content-length', 0))
      ret_val = True

      with open(file_name, mode) as f:
        with tqdm(
            total=file_size, unit='B', unit_scale=True, 
            unit_divisor=1024, desc=file_name, initial=initial_pos, 
            ascii=True, miniters=1) as pbar:

              for chunk in response.iter_content(64 * block_size):
                  f.write(chunk)
                  pbar.update( len(chunk) )

  return ret_val
    


In [7]:
def unzip_file( substr_name , name, file_ext='.zip', file_path='./' ):
  #if doesn't exist one folder in the first label, create one with the name
  from zipfile import ZipFile, LargeZipFile

  #Get folder name
  f_name = ''
  for dirname, _, filenames in os.walk('./'): 

    for filename in filenames:   
      if ( substr_name in filename and os.path.splitext(filename)[1] == file_ext ):
          f_name = filename
          break

    print(f_name)
    try:
      with ZipFile( f_name, 'r') as compress_file:
        
        # Create a folder when .zip file is found
        try:
          os.mkdir(name)
        except:
          print("Dir {} exists!".format(name) )

        print(compress_file)
          
        # Extract all the contents of zip file in directory with the name: name
        # contents = compress_file.read()
        compress_file.extractall(name)

    except LargeZipFile:
      print("Error: File size if too large: {}.".format(substr_name) )
      # print("Cannot find the zip file for: {}.".format(substr_name) )
      f_name = ''

    return f_name

# Image collection

In [8]:
download_zip_file("https://github.com/OlafenwaMoses/IdenProf/releases/download/v1.0/idenprof-jpg.zip", "idenprof-jpg.zip")


idenprof-jpg.zip: 100%|##########| 148M/148M [00:05<00:00, 27.0MB/s]


True

In [9]:
unzip_file("idenprof-jpg", "")

idenprof-jpg.zip
Dir  exists!
<zipfile.ZipFile filename='idenprof-jpg.zip' mode='r'>


'idenprof-jpg.zip'

# Create the model

In [11]:
from imageai.Prediction.Custom import ModelTraining

model_trainer = ModelTraining()
model_trainer.setModelTypeAsResNet()
model_trainer.setDataDirectory("idenprof")
model_trainer.trainModel(num_objects=10, num_experiments=10, enhance_data=True, batch_size=64, show_network_summary=False)

Using Enhanced Data Generation
Found 9000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
JSON Mapping for the model classes saved to  idenprof/json/model_class.json
Number of experiments (Epochs) :  10
Epoch 1/10
 31/140 [=====>........................] - ETA: 32s - loss: 5.8269 - acc: 0.0610
Epoch 00001: val_acc improved from -inf to 0.06099, saving model to idenprof/models/model_ex-001_acc-0.060988.h5
140/140 [==============================] - 207s 1s/step - loss: 1.9906 - acc: 0.3852 - val_loss: 5.8269 - val_acc: 0.0610
Epoch 2/10
 31/140 [=====>........................] - ETA: 24s - loss: 13.9994 - acc: 0.2560
Epoch 00002: val_acc improved from 0.06099 to 0.25605, saving model to idenprof/models/model_ex-002_acc-0.256048.h5
140/140 [==============================] - 138s 987ms/step - loss: 1.6193 - acc: 0.4552 - val_loss: 13.9994 - val_acc: 0.2560
Epoch 3/10
 31/140 [=====>........................] - ETA: 24s - loss: 2.4969 - acc: 0.3125
Epoch 00003: val

# Predict

In [13]:
from imageai.Prediction.Custom import CustomImagePrediction

prediction = CustomImagePrediction()
prediction.setModelTypeAsResNet()
prediction.setModelPath("idenprof/models/model_ex-010_acc-0.701109.h5") #/content/idenprof/models/model_ex-010_acc-0.701109.h5
prediction.setJsonPath("idenprof/json/model_class.json")
prediction.loadModel(num_objects=10)

In [18]:
predictions, probabilities = prediction.predictImage("_waiter_.jpg", result_count=3)

for eachPrediction, eachProbability in zip(predictions, probabilities):
    print(eachPrediction , " : " , eachProbability)

waiter  :  42.808035016059875
pilot  :  29.5640766620636
chef  :  11.947136372327805
